In [15]:
#%tb
import os
import json
#import argparse
import numpy as np
from utils.dataLoaders import load_glassdoor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
#from configs import system_configs
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from utils.dataLoaders import clean_str
from source.torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from sklearn.ensemble import RandomForestClassifier
import source.utils

In [3]:
#configs of the experiment
ID            = None#= system_configs.ID #name of experiment
dataset    = 'amazon'   #= system_configs.dataset #dataset(s) we are using example: "amazon_microsoft" means using both amazon and microsoft data
data_source  = 'glassdoor' #= system_configs.data_source #data source to run, only glassdoor for now
batch_size   = 256 #= system_configs.batch_size #batch size
learning_rate = 0.001#= system_configs.learning_rate
mode        = 'train'   #= system_configs.mode #train val test
classifier   = 'cnn' #= system_configs.classifier #type of classifier
word_embedding = 'GloVe'#= system_configs.word_embedding#word embedding: GloVe/Bert
rating_mode = 2#= system_configs.rating_mode #binary, tenary or 5 star rating
rebalance  = False#= system_configs.rebalance

GLOVE_HOME = os.path.join('data', 'glove.6B')
glove_lookup = source.utils.glove2dict(os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [22]:
def get_revirew_embed(reviews, lookup, func=np.sum):
    result = []
    for r in reviews:
        embed = np.array([lookup[w] for w in r if w in lookup]) 
        if len(embed) == 0:
            dim = len(next(iter(lookup.values())))
            e = np.zeros(dim)
        else:
            e = func(embed,  axis = 0)
        result.append(e)
    return np.array(result)

def torch_pros_cons():
    X, y = load_glassdoor("all","train", 2, "pros_cons", labelType="pros_cons")
    test_X, test_y = load_glassdoor("collections", "train", 2, "pros_cons", labelType = "pros_cons", restrict = 20000)
    embed_X = get_revirew_embed(X, glove_lookup, np.mean)
    embed_test_X = get_revirew_embed(test_X, glove_lookup, np.mean)
    clf = RandomForestClassifier(max_features = 200, min_samples_split = 10, n_estimators = 10, oob_score = True,
                             max_depth = 20, criterion = 'entropy', warm_start = True, random_state = 5)
    print(embed_X.shape, len(y))
    clf.fit(embed_X, y)
    report_pros = classification_report(clf.predict(embed_test_X), test_y)
    print(report_pros)
    

In [23]:
torch_pros_cons()

(20027, 300) 20027
              precision    recall  f1-score   support

           0       0.67      0.65      0.66     10294
           1       0.64      0.66      0.65      9706

    accuracy                           0.65     20000
   macro avg       0.65      0.65      0.65     20000
weighted avg       0.65      0.65      0.65     20000



/Users/baidi/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/Users/baidi/anaconda3/envs/nlu/lib/python3.7/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
